# AI Mathematical Olympiad - Progress Prize 3
# ゴール指向の逆算推論（Backward Chaining）アプローチ

## 概要
このNotebookは、数学オリンピック問題をAIで解くための**ゴール指向アプローチ**を実装します。

### 従来手法（多数決）との違い
- ❌ 従来: 手当たり次第に解を生成し、多数決で答えを決める
- ✅ 本手法: 問題から「何を求めるか」を逆算し、計画的に解く

### フェーズ構成
1. **Phase 1**: 問題分析（Goal/Given/分野の抽出）
2. **Phase 2**: 逆算推論（サブゴール分解）
3. **Phase 3**: 計画実行（Pythonコード生成・実行）
4. **Phase 4**: 検証・提出

---
## 0. セットアップ

In [ ]:
import pandas as pd
import numpy as np
import json
import re

# Kaggle環境でのデータパス
DATA_PATH = '/kaggle/input/ai-mathematical-olympiad-progress-prize-3/'

# データ読み込み
df = pd.read_csv(DATA_PATH + 'reference.csv')
print(f'問題数: {len(df)}')
df.head()

---
## 1. Phase 1: 問題分析モジュール

問題文を構造化データに変換する。
- **Goal**: 最終的に求めるもの
- **Given**: 与えられた条件
- **Field**: 問題の分野（代数/幾何/整数論/組合せ）

In [ ]:
# 問題分析用プロンプトテンプレート
PROBLEM_ANALYSIS_PROMPT = '''
あなたは数学オリンピックの問題を分析する専門家です。
以下の数学問題を分析し、JSON形式で構造化してください。

## 問題文
{problem}

## 出力形式（JSON）
```json
{{
    "goal": "最終的に求めるもの（例: 面積、個数、余り、最大値など）",
    "goal_expression": "数式で表現したゴール（例: abc mod 10^5）",
    "given": [
        "与えられた条件1",
        "与えられた条件2"
    ],
    "field": "algebra | geometry | number_theory | combinatorics | mixed",
    "key_concepts": ["使用する可能性のある定理や概念"]
}}
```

JSONのみを出力してください。
'''

def create_analysis_prompt(problem_text: str) -> str:
    """問題文から分析プロンプトを生成"""
    return PROBLEM_ANALYSIS_PROMPT.format(problem=problem_text)

def parse_analysis_response(response: str) -> dict:
    """LLMの応答からJSONを抽出してパース"""
    json_match = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
    else:
        json_str = response.strip()
    
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        return {"error": f"JSON parse error: {e}", "raw": response}

In [ ]:
# 1問目でプロンプト生成テスト
problem_text = df.iloc[0]['problem']
print('=== 問題文 ===')
print(problem_text)
print('\n=== 生成されたプロンプト ===')
prompt = create_analysis_prompt(problem_text)
print(prompt)

---
## 2. Phase 2: 逆算推論モジュール

ゴールから逆算してサブゴールを分解する。

（TODO: LLMを使った実装）

In [ ]:
# Phase 2: 逆算推論プロンプトテンプレート
BACKWARD_REASONING_PROMPT = '''
あなたは数学オリンピックの問題を解く専門家です。
以下の問題分析結果をもとに、ゴールから逆算して解法計画を立ててください。

## 問題分析
{analysis}

## 指示
1. ゴールを達成するために必要なサブゴールを列挙
2. 各サブゴールをさらに分解
3. 与えられた条件から計算可能なステップまで落とし込む

## 出力形式（JSON）
```json
{{
    "goal": "最終ゴール",
    "subgoals": [
        {{
            "step": 1,
            "description": "サブゴール1の説明",
            "requires": ["必要な中間値"]
        }}
    ],
    "execution_order": ["計算順序: 下から上へ"]
}}
```
'''

# TODO: LLM呼び出し実装

---
## 3. Phase 3: 計画実行モジュール

解法計画をPythonコードに変換し実行する。

（TODO: LLMを使った実装）

In [ ]:
# Phase 3: コード生成プロンプト
CODE_GENERATION_PROMPT = '''
以下の解法計画に基づいて、Pythonコードを生成してください。

## 解法計画
{plan}

## 要件
- 最終的な答えを `answer` 変数に格納
- 必要に応じて sympy を使用
- 計算過程をコメントで説明

```python
# ここにコードを生成
```
'''

def execute_code_safely(code: str, timeout: int = 30) -> dict:
    """生成されたコードを安全に実行"""
    import subprocess
    import tempfile
    
    with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
        f.write(code)
        f.write('\nprint(f"ANSWER={answer}")')
        temp_path = f.name
    
    try:
        result = subprocess.run(
            ['python', temp_path],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        output = result.stdout
        error = result.stderr
        
        # 答えを抽出
        answer_match = re.search(r'ANSWER=(\d+)', output)
        answer = int(answer_match.group(1)) if answer_match else None
        
        return {'success': True, 'answer': answer, 'output': output, 'error': error}
    except subprocess.TimeoutExpired:
        return {'success': False, 'error': 'Timeout'}
    except Exception as e:
        return {'success': False, 'error': str(e)}

---
## 4. Phase 4: 検証・提出

（TODO: reference.csvでの評価、Kaggle API形式での提出）

In [ ]:
# Phase 4: 評価関数
def evaluate_on_reference(solve_fn, df):
    """reference.csvで正答率を計算"""
    correct = 0
    total = len(df)
    
    for idx, row in df.iterrows():
        problem = row['problem']
        expected = row['answer']
        
        predicted = solve_fn(problem)
        
        if predicted == expected:
            correct += 1
            print(f'✅ Problem {idx}: Correct ({expected})')
        else:
            print(f'❌ Problem {idx}: Expected {expected}, Got {predicted}')
    
    print(f'\n正答率: {correct}/{total} ({100*correct/total:.1f}%)')
    return correct / total

---
## 次のステップ

1. LLMモデルをロード（Gemma / DeepSeek / Qwen）
2. Phase 1-3を統合したパイプラインを完成
3. reference.csvで評価
4. Kaggle提出形式に変換